<a href="https://colab.research.google.com/github/DavidAlba2627/quantum-budget-optimization/blob/main/Quantum_Optimization_Marketing_Budget_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install qiskit libraries

In [ ]:
!pip install qiskit==1.4.1
!pip install qiskit-algorithms
!pip install qiskit-optimization

In [ ]:
import qiskit, qiskit_algorithms, qiskit_optimization
print("Qiskit:", qiskit.__version__)
print("Qiskit Algorithms:", qiskit_algorithms.__version__)
print("Qiskit Optimization:", qiskit_optimization.__version__)

Qiskit: 1.4.1
Qiskit Algorithms: 0.3.1
Qiskit Optimization: 0.6.1


In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from qiskit_algorithms.minimum_eigensolvers import QAOA, SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

# 📊 Integer Quadratic Optimization Problem – Marketing Budget Allocation

We aim to allocate a limited advertising budget across three distinct customer segments to **maximize total utility**, considering:

- The **individual return** of each segment
- The **saturation effect** (diminishing returns) when investing too much in one segment
- The **interaction effects** (synergies or cannibalization) between segments


## 🧠 Decision Variables

Let:

- $x_1$ : Budget allocated to **Children**
- $x_2$ : Budget allocated to **Single Adults**
- $x_3$ : Budget allocated to **Parents**

Each variable represents the number of **budget blocks** in units of $\$1000$, and is restricted to integer values between 0 and 5 ($\$0$ to $\$5000$).

## 📈 Objective Function: Total Utility

We want to **maximize total utility**, defined as:

$$\begin{align}
f(x) = \sum_{i=1}^{3} a_i x_i - \sum_{i=1}^{3} b_i x_i^2 + \sum_{i<j} c_{ij} x_i x_j
\end{align}$$


With the specific values:

- $a = [6.5, 9.0, 7.8]$ — base return per budget unit for each segment

- $b = [0.6, 0.9, 0.5]$ — saturation penalties (quadratic)

- $c = \begin{bmatrix}
0 & 0.3 & -0.4 \\
0.3 & 0 & 0.7 \\
-0.4 & 0.7 & 0 \\
\end{bmatrix}$ — interaction coefficients between segments


So the function becomes:

$$\begin{align}
\text{Maximize}\quad{} f(x) = 6.5x_1 + 9.0x_2 + 7.8x_3 - (0.6x_1^2 + 0.9x_2^2 + 0.5x_3^2) + 0.3x_1x_2 - 0.4x_1x_3 + 0.7x_2x_3
\end{align}$$

## 🔒 Final Optimization Model

We solve the following integer quadratic programming problem:

$$\begin{align}
\text{Maximize}\quad{} & f(x) = 6.5x_1 + 9.0x_2 + 7.8x_3 - (0.6x_1^2 + 0.9x_2^2 + 0.5x_3^2) + 0.3x_1x_2 - 0.4x_1x_3 + 0.7x_2x_3\\
\text{subject to}\quad{} & x_1 + x_2 + x_3 \leq 10   \\
                         & 0 \leq x_1 \leq 5         \\
                         & 0 \leq x_2 \leq 5         \\
                         & 0 \leq x_3 \leq 5         \\
\end{align}$$

**By solving this model, we obtain the optimal allocation of advertising budget that maximizes the total expected gain while avoiding over-saturation and leveraging segment interactions.**

# 🎯 Solving the problem

## Defining the quadratic problem using `QuadraticProgram` from `qiskit_optimization`

In [ ]:
def budget_allocation_quadratic_program():
    # Creating the quadratic model
    budget_allocation = QuadraticProgram(name="Marketing Budget Allocation")
    # Defining variables and ranges of values (constraints)
    budget_allocation.integer_var(name="x1", lowerbound=0, upperbound=5)
    budget_allocation.integer_var(name="x2", lowerbound=0, upperbound=5)
    budget_allocation.integer_var(name="x3", lowerbound=0, upperbound=5)
    # Defining the function to optimize
    budget_allocation.maximize(
        linear={"x1": 6.5, "x2": 9, "x3": 7.8},
        quadratic={("x1", "x1"): -0.6, ("x2", "x2"): -0.9, ("x3", "x3"): -0.5,
         ("x1", "x2"): 0.3,  ("x1", "x3"): -0.4, ("x2", "x3"): 0.7},
    )
    # Defining the remaining constraints
    budget_allocation.linear_constraint(linear={"x1": 1, "x2": 1, "x3": 1}, sense="<=", rhs=10)
    return budget_allocation

Problem Summary

In [ ]:
print(budget_allocation_quadratic_program().export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Marketing Budget Allocation

Maximize
 obj: 6.500000000000 x1 + 9 x2 + 7.800000000000 x3 + [ - 1.200000000000 x1^2
      + 0.600000000000 x1*x2 - 0.800000000000 x1*x3 - 1.800000000000 x2^2
      + 1.400000000000 x2*x3 - x3^2 ]/2
Subject To
 c0: x1 + x2 + x3 <= 10

Bounds
       x1 <= 5
       x2 <= 5
       x3 <= 5

Generals
 x1 x2 x3
End



### Convert to QuadraticPrograms

To estimate how many qubits this quadratic program requires, we convert it to an **Ising model** and print the parameter `num_qubits`. An **Ising model** is a special type of system model that is well-suited to quantum computing.

Although we won't be using a specific model, the algorithms and classes in `Qiskit` we use internally perform this conversion.

In [ ]:
# Estimate the number of qubits needed for the quadratic problem
budget_allocation = budget_allocation_quadratic_program()
ising_operations, _ = (
    QuadraticProgramToQubo().convert(budget_allocation,).to_ising()
)
print(f"The number of qubits needed is: {ising_operations.num_qubits}")

The number of qubits needed is: 13


## Ways to solve the problem

There are three ways to run a quantum algorithm using `Qiskit`:
1. In a local simulator on your own computer
2. In a cloud-hosted simulator hosted by **IBM**
3. On a real quantum computer accessible through **IBM Quantum**

All of these are called backends. In all cases, the backend can be easily changed for another as long as the simulator or device has the appropriate resources (number of qubits, etc.)

We would like to compare our quantum solution with the classical one. The following subsections show how these different methods are implemented to solve the Marketing Budget Allocation problem. The method used for the Quantum solution is the **Quantum Approximate Optimization Algorithm (QAOA)**

### Classical solution

The classic solution for this problem can be easily found using `NumPy` and `Qiskit`. The **QUBO problem** can be solved by finding the **minimum eigenvalue** of its underlying **matrix representation**. Fortunately, we don't need to know what this matrix looks like. We just need to pass it to `MinimumEigensolver` and `MinimumEigenOptimizer`

The optimizer translates the provided problem into a parameterized representation, which is then passed to the solver. By optimizing the parameters, the solver will eventually yield the minimum eigenvalue for the parameterized representation and, therefore, the solution to the original problem. Here we use a classic `NumPy` solver, the `NumPyMinimumEigensolver`

In [ ]:
def get_classical_solution(quadratic_program):
    # Create the solver
    solver = NumPyMinimumEigensolver()
    # Create the optimizer for the solver
    optimizer = MinimumEigenOptimizer(solver)
    # Obtain the optimizer result
    return optimizer.solve(quadratic_program)

If we run the classical method for our problem, we obtain a valid solution that maximizes utility

In [ ]:
# Obtain classical result
classical_result = get_classical_solution(budget_allocation)
# Print the result for maximum utility
print("##### Solution found using the classical method #####\n")
print(f"The maximum utility is: ${classical_result.fval}k \n")
# Print the results for each segment
print(f"The budget allocated for each segment is: ")
_segments = [v.name for v in budget_allocation.variables]
for segment_id, budget in enumerate(classical_result.x):
    print(f"-- ${int(budget)}k for {_segments[segment_id]}")

##### Solution found using the classical method #####

The maximum utility is: $67.2k 

The budget allocated for each segment is: 
-- $1k for x1
-- $4k for x2
-- $5k for x3


### QAOA Solution

To solve our problem using **QAOA**, we just need to replace the classical solver with an instance of the `QAOA` class. Now that we are running a quantum algorithm, we need to tell the solver where to execute the quantum component. This time we are using an instance of `Sampler`. **QAOA** is an iterative algorithm and run multiple times with different internal parameters. The parameters are classically tuned during the optimization step of the computation using the `optimizer`, for which we will use `COBYLA`. To determine how many iterations there are, we define a callback function that runs for each iteration and stores the number of evaluations so far. At the end of our algorithm's execution, we return the result and the number of iterations.

In [ ]:
def get_QAOA_solution_for(quadratic_program, sampler, base_optimizer):
    # Counter for evaluation iterations
    _eval_count = 0
    # Callback function
    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create the solver
    qaoa_solver = QAOA(sampler, optimizer=base_optimizer, callback=callback)
    # Create the optimizer for the solver
    eigen_optimizer  = MinimumEigenOptimizer(qaoa_solver)
    # Obtain the optimizer result
    result = eigen_optimizer.solve(quadratic_program)
    return result, _eval_count

In [ ]:
# Obtain result using QAOA
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(budget_allocation, Sampler(), COBYLA())

# Print the result for maximum utility
print("##### Solution found using QAOA method #####\n")
print(f"The maximum utility is: ${qaoa_result.fval}k \n")
# Print the results for each segment
print(f"The budget allocated for each segment is: ")
for budget, segment in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"-- ${int(budget)}k for {segment}")

print(f"\nSolution obtained in {qaoa_eval_count} QAOA evaluations")

##### Solution found using QAOA method #####

The maximum utility is: $67.2k 

The budget allocated for each segment is: 
-- $1k for x1
-- $4k for x2
-- $5k for x3

Solution obtained in 27 QAOA evaluations
